In [1]:
import pandas as pd
import numpy as np
import os, glob
import pandas_plink as pdp
import scipy.stats as stats
from scipy.stats import fisher_exact


In [2]:
GWAS_RESULTS_PATH = "/cs/labs/michall/roeizucker/IIH/GWAS/results"
PLINK_RSULTS_FILE_PATTERN = "*.glm.*"
PLINK_FILE_BASE_PATH = "/cs/labs/michall/roeizucker/plink_results/reduced_snps2/small_ch"
PROJECT_PATH = "/cs/labs/michall/roeizucker/SKAT_experiments/SetID"
FILE_CONTAINING_PARTICIPANTS = PROJECT_PATH + "/data_participant.csv"
OLD_PHENOTYPE_FILE = '/cs/labs/michall/roeizucker/10krun/runs/48:59/ukbb_dataset.csv'

OR_TEST_THREASHOLD = 20

In [3]:
old_phenotype_dataframe = pd.read_csv(OLD_PHENOTYPE_FILE)

participants_dataframe = pd.read_csv(FILE_CONTAINING_PARTICIPANTS)
participants_dataframe["IIH_mod"] = 1
participants_dataframe["eid2"] = participants_dataframe["eid"]
participants_dataframe.set_index("eid2",inplace=True)
print(len(old_phenotype_dataframe))
old_phenotype_dataframe = pd.read_csv(OLD_PHENOTYPE_FILE)
old_phenotype_dataframe["eid2"] = old_phenotype_dataframe["eid"]
old_phenotype_dataframe.set_index("eid2",inplace=True)
old_phenotype_dataframe["IIH_mod"] = participants_dataframe["IIH_mod"]
old_phenotype_dataframe["IIH_mod"] = old_phenotype_dataframe["IIH_mod"].fillna(0)
old_phenotype_dataframe["eid"] = old_phenotype_dataframe["eid"].astype(str)
print(len(old_phenotype_dataframe))


333415
333415


In [4]:
dataframes = []
for filename in glob.glob(os.path.join(GWAS_RESULTS_PATH, PLINK_RSULTS_FILE_PATTERN)):
#     with open(os.path.join(os.getcwd(), filename), 'r') as f: # open in readonly mode
#         val = f.read()
    df = pd.read_csv(filename,sep="\t")
    dataframes.append(df)
gwas_results = pd.concat(dataframes)
# gwas_results
gwas_results_not_na = gwas_results[gwas_results["OR"].notna()]

#             print(val)

In [5]:
# gwas_results_filtered = gwas_results_not_na[gwas_results_not_na["OR"] > OR_TEST_THREASHOLD]
gwas_results_filtered = gwas_results_not_na
gwas_results_filtered = gwas_results_filtered[gwas_results_filtered["P"] < 10**-1]
chrom_vals = set(gwas_results_filtered["#CHROM"])
gwas_results_filtered["OR"].describe()

count    23570.000000
mean        18.205441
std         68.666090
min          0.089416
25%          2.789343
50%          7.154925
75%         15.689225
max       2754.770000
Name: OR, dtype: float64

In [67]:
# df["eid2"] = df["iid"]
# df["eid"] = df["iid"]

# # df["a"] = old_phenotype_dataframe["sample_index"]
# new_df = df.merge(old_phenotype_dataframe, on='eid', how='left').dropna(subset=["IIH_mod_y"])
# # df["IIH_mod"]

In [6]:
# df
gwas_results_filtered["new_OR"] = -1
gwas_results_filtered["new_P"] = -1


/cs/usr/roeizucker/my_python/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/cs/usr/roeizucker/my_python/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
for chrom in chrom_vals:
    chrom_specific = gwas_results_filtered[gwas_results_filtered["#CHROM"] == chrom]
#     df = pdp.read_plink(PLINK_FILE_BASE_PATH + str(chrom),verbose=False)[0][["snp","pos"]]
    base_path = PLINK_FILE_BASE_PATH + str(chrom)
    bim_path = base_path + ".bim"
    bed_path = base_path + ".bed"
    fam_path = base_path + ".fam"
    G = pdp.read_plink1_bin(bed_path,bim_path,fam_path,verbose=False)
    all_variants_list = list(G[0]["snp"].values)
    requested_variants = list(chrom_specific["ID"])
    for var in requested_variants:
        val_loc = list(G[0]["snp"].values).index(var)
        specific = G.T[val_loc]
        df = specific.to_dataframe()
        df["eid"] = df["iid"]
        new_df = df.merge(old_phenotype_dataframe, on='eid', how='left').dropna(subset=["IIH_mod"])
#         a = len(new_df[(new_df["IIH_mod"] == 1) & new_df["genotype"]!=0])
        a = len(new_df.query("IIH_mod == 1 & genotype != 0"))
#         b = len(new_df[(new_df["IIH_mod"] == 0) & new_df["genotype"]!=0])
        b = len(new_df.query("IIH_mod == 0 & genotype != 0"))
#         c = len(new_df[(new_df["IIH_mod"] == 1) & new_df["genotype"]==0])
        c = len(new_df.query("IIH_mod == 1 & genotype == 0"))
#         d = len(new_df[(new_df["IIH_mod"] == 0) & new_df["genotype"]==0])
        d = len(new_df.query("IIH_mod == 0 & genotype == 0"))
        
        odds_ratio = (a*d)/(b*c)
        print("odds_ratio",odds_ratio)
        print("1/odds_ratio",1/odds_ratio)
        table = [[a, b], [c, d]]
        res = fisher_exact(table, alternative='two-sided')
        print("fisher_exact",res)
        print(table)
        print(chrom_specific[chrom_specific["ID"] == var])
        gwas_results_filtered[gwas_results_filtered["ID"] == var]["new_OR"] = res[0]
        gwas_results_filtered[gwas_results_filtered["ID"] == var]["new_P"] = res[1]
        print(gwas_results_filtered[gwas_results_filtered["ID"] == var])
        input()
#         print(a)
#         input()



#     genotype, IIH_mod
# odds ratio calculation:
#                  Event (IIH_mod)
#                Yes    No
# Exposore  Yes  a      b
#(genotype) No   c      d
# OR = ad/bc
#     print(G) 

/cs/usr/roeizucker/my_python/lib/python3.7/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/cs/usr/roeizucker/my_python/lib/python3.7/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


odds_ratio 4.2903793406820885
1/odds_ratio 0.2330796231740709
fisher_exact (4.2903793406820885, 0.08132401385411835)
[[2, 904], [171, 331612]]
    #CHROM     POS           ID REF ALT A1 TEST    OBS_CT       OR  \
24       1  874467  rs754562511   A   G  A  ADD  214654.0  15.9628   
24       1  874467  rs754562511   A   G  A  ADD  273562.0  14.5169   

    LOG(OR)_SE   Z_STAT         P  new_OR  new_P  
24     1.02304  2.70788  0.006771      -1     -1  
24     1.00976  2.64947  0.008062      -1     -1  
    #CHROM     POS           ID REF ALT A1 TEST    OBS_CT       OR  \
24       1  874467  rs754562511   A   G  A  ADD  214654.0  15.9628   
24       1  874467  rs754562511   A   G  A  ADD  273562.0  14.5169   

    LOG(OR)_SE   Z_STAT         P  new_OR  new_P  
24     1.02304  2.70788  0.006771      -1     -1  
24     1.00976  2.64947  0.008062      -1     -1  


KeyboardInterrupt: Interrupted by user

In [43]:
! ls -la /cs/labs/michall/roeizucker/plink_results/reduced_snps2 | head

total 76757061
drwxr-sr-x 2 roeizucker proto         98 Dec 13  2021 .
drwxr-sr-x 7 roeizucker proto         11 Nov  3  2021 ..
-rw-r--r-- 1 roeizucker proto    7644763 Nov  4  2021 all_variants.txt
-rw-r--r-- 1 roeizucker proto        555 Dec 13  2021 plink2.log
-rwxr--r-- 1 roeizucker proto       1937 Nov 14  2021 runner.sh
-rw-r--r-- 1 roeizucker proto 2992100418 Nov 13  2021 small_ch10.bed
-rw-r--r-- 1 roeizucker proto     743667 Nov 13  2021 small_ch10.bim
-rw-r--r-- 1 roeizucker proto   12185095 Nov 13  2021 small_ch10.fam
-rw-r--r-- 1 roeizucker proto       1293 Nov 13  2021 small_ch10.log


In [8]:
G

<xarray.DataArray 'genotype' (sample: 487409, variant: 64451)>
dask.array<transpose, shape=(487409, 64451), dtype=float32, chunksize=(1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/14)
  * sample   (sample) object '1688461' '5444187' ... '1164524' '1409864'
  * variant  (variant) <U12 'variant0' 'variant1' ... 'variant64450'
    fid      (sample) object '1688461' '5444187' ... '1164524' '1409864'
    iid      (sample) object '1688461' '5444187' ... '1164524' '1409864'
    father   (sample) object '0' '0' '0' '0' '0' '0' ... '0' '0' '0' '0' '0' '0'
    mother   (sample) object '0' '0' '0' '0' '0' '0' ... '0' '0' '0' '0' '0' '0'
    ...       ...
    chrom    (variant) object '1' '1' '1' '1' '1' '1' ... '1' '1' '1' '1' '1'
    snp      (variant) object 'rs568964432' 'rs140739101' ... 'rs139567720'
    cm       (variant) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    pos      (variant) int32 69224 69428 69487 ... 249212272 249212393 249212402
    a0       (variant) object 'A' 'T' 'G' 'G' 'T' 'A' ... 'G' 'G' 'A' 'G' 'G'
    a1       (variant) object 'T' 'G' 'A' 'A' 'A' 'G' ... 'C' 'A' 'G' 'A' 'A'